Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [80]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import time

First reload the data we generated in _notmist.ipynb_.

In [11]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [13]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Logistic Regression

In [34]:
batch_size = 128
l2_value  = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 

  # L2 regularization for the fully connected parameters.
  loss += l2_value * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.650734
Minibatch accuracy: 9.4%
Validation accuracy: 11.3%
Minibatch loss at step 500: 0.915404
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 0.728188
Minibatch accuracy: 79.7%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 0.815118
Minibatch accuracy: 80.5%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.755349
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.702102
Minibatch accuracy: 84.4%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.646210
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.3%


#### MLP

In [44]:
batch_size = 128
l2_value = 0.01
n_hidden_nodes= 1024 # relu layer

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Training computation.
  
  # Hidden layer 1
  weights_relu = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
  biases_relu = tf.Variable(tf.zeros([n_hidden_nodes]))
    
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights_relu) + biases_relu)
    
  # linear layer 
  weights = tf.Variable(tf.truncated_normal([n_hidden_nodes,num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))    

  logits = tf.matmul(relu_layer, weights) + biases

  # loss/cost
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_relu))
  loss += l2_value * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_relu) + biases_relu), weights) + biases 
    )
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_relu) + biases_relu), weights) + biases
    )  

In [45]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    
    choice = np.random.choice(train_labels.shape[0],batch_size)
    batch_data = train_dataset[choice, :]
    batch_labels = train_labels[choice, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3501.701660
Minibatch accuracy: 13.3%
Validation accuracy: 20.2%
Minibatch loss at step 2000: 424.398804
Minibatch accuracy: 94.5%
Validation accuracy: 81.1%
Minibatch loss at step 4000: 57.700481
Minibatch accuracy: 97.7%
Validation accuracy: 84.3%
Minibatch loss at step 6000: 8.375528
Minibatch accuracy: 89.1%
Validation accuracy: 85.3%
Minibatch loss at step 8000: 1.632802
Minibatch accuracy: 91.4%
Validation accuracy: 85.0%
Minibatch loss at step 10000: 0.777581
Minibatch accuracy: 85.9%
Validation accuracy: 85.0%
Minibatch loss at step 12000: 0.723613
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 14000: 0.779479
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 16000: 0.497736
Minibatch accuracy: 91.4%
Validation accuracy: 84.8%
Minibatch loss at step 18000: 0.545747
Minibatch accuracy: 91.4%
Validation accuracy: 84.9%
Minibatch loss at step 20000: 0.757975
Minibatch accuracy: 81.2%
Valid

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [47]:
batch_size = 128
n_hidden_nodes= 1024 # relu layer

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Training computation.
  
  # Hidden layer 1
  weights_relu = tf.Variable( tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
  biases_relu = tf.Variable(tf.zeros([n_hidden_nodes]))
    
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights_relu) + biases_relu)
    
  # linear layer 
  weights = tf.Variable(tf.truncated_normal([n_hidden_nodes,num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))    

  logits = tf.matmul(relu_layer, weights) + biases

  # loss/cost
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_relu) + biases_relu), weights) + biases 
    )
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_relu) + biases_relu), weights) + biases
    )  

In [54]:
num_steps = 3001

n_small_set = 10*batch_size

train_dataset_small = train_dataset[:n_small_set,:]
train_labels_small  = train_labels[:n_small_set,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.    
    choice = np.random.choice(train_labels_small.shape[0],batch_size)
    batch_data = train_dataset_small[choice, :]
    batch_labels = train_labels_small[choice, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 320.449982
Minibatch accuracy: 7.0%
Validation accuracy: 18.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.7%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 2500: 0.000003
Minibatch accuracy: 100.0%
Validation accuracy: 71.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Test accuracy: 78.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [56]:
batch_size = 128
l2_value = 0.01
n_hidden_nodes= 1024 # relu layer
dropout= 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Training computation.
  
  # Hidden layer 1 
  weights_relu = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
  biases_relu = tf.Variable(tf.zeros([n_hidden_nodes]))
    
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights_relu) + biases_relu)

  # Apply Dropout
  relu_layer = tf.nn.dropout(relu_layer, dropout)
  
    
  # linear layer 
  weights = tf.Variable(tf.truncated_normal([n_hidden_nodes,num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))    

  logits = tf.matmul(relu_layer, weights) + biases

  # loss/cost
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_relu))
  loss += l2_value * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_relu) + biases_relu), weights) + biases 
    )
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_relu) + biases_relu), weights) + biases
    )  

In [57]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    choice = np.random.choice(train_labels.shape[0],batch_size)
    batch_data = train_dataset[choice, :]
    batch_labels = train_labels[choice, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3636.136230
Minibatch accuracy: 10.9%
Validation accuracy: 16.9%
Minibatch loss at step 2000: 418.562622
Minibatch accuracy: 88.3%
Validation accuracy: 83.9%
Minibatch loss at step 4000: 56.743553
Minibatch accuracy: 92.2%
Validation accuracy: 85.1%
Minibatch loss at step 6000: 8.178691
Minibatch accuracy: 89.1%
Validation accuracy: 85.0%
Minibatch loss at step 8000: 1.717211
Minibatch accuracy: 88.3%
Validation accuracy: 84.9%
Minibatch loss at step 10000: 0.890289
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 12000: 0.600429
Minibatch accuracy: 92.2%
Validation accuracy: 84.7%
Minibatch loss at step 14000: 0.740104
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 16000: 0.709455
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
Minibatch loss at step 18000: 0.586536
Minibatch accuracy: 91.4%
Validation accuracy: 84.8%
Minibatch loss at step 20000: 0.655677
Minibatch accuracy: 89.1%
Valid

#### Overfitting case with dropout

In [58]:
num_steps = 3001

n_small_set = 10*batch_size

train_dataset_small = train_dataset[:n_small_set,:]
train_labels_small  = train_labels[:n_small_set,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.    
    choice = np.random.choice(train_labels_small.shape[0],batch_size)
    batch_data = train_dataset_small[choice, :]
    batch_labels = train_labels_small[choice, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3666.666504
Minibatch accuracy: 11.7%
Validation accuracy: 21.2%
Minibatch loss at step 500: 1901.410400
Minibatch accuracy: 90.6%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 1150.650879
Minibatch accuracy: 94.5%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 697.111877
Minibatch accuracy: 96.1%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 422.430328
Minibatch accuracy: 98.4%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 256.175690
Minibatch accuracy: 98.4%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 155.325882
Minibatch accuracy: 100.0%
Validation accuracy: 81.2%
Test accuracy: 87.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [307]:
batch_size = 128
l2_value = 0.002
n_hidden_1, n_hidden_2, n_hidden_3= 512, 512, 256

dropout= 0.70
learning_rate, decay_rate, decay_steps = 0.002, 0.9, 300000

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Store layers weight & bias
  weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1])),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
        'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
        'out':tf.Variable(tf.truncated_normal([n_hidden_3,num_labels]))
  }

  biases = {
        'h1': tf.Variable(tf.zeros([n_hidden_1])),
        'h2': tf.Variable(tf.zeros([n_hidden_2])),
        'h3': tf.Variable(tf.zeros([n_hidden_3])),
        'out': tf.Variable(tf.zeros([num_labels]))  
  }  
  
  # Hidden layer 1 
  hidden_layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights['h1']) + biases['h1'])
  hidden_layer_1 = tf.nn.dropout(hidden_layer_1, dropout) # Apply Dropout
  
  # Hidden layer 2 
  hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1,weights['h2']) + biases['h2'])
  hidden_layer_2 = tf.nn.dropout(hidden_layer_2, dropout) # Apply Dropout
  
  # Hidden layer 3 
  hidden_layer_3 = tf.nn.relu(tf.matmul(hidden_layer_2,weights['h3']) + biases['h3'])
  hidden_layer_3 = tf.nn.dropout(hidden_layer_3, dropout) # Apply Dropout  
    
  # linear layer 
  logits = tf.matmul(hidden_layer_3, weights['out']) + biases['out']

  # loss/cost
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = sum([tf.nn.l2_loss(weights[l]) for l in weights.keys()])
  loss += l2_value * regularizers
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)


  def predict(dataset,weights,biases):
        layer1 = tf.nn.relu(tf.matmul(dataset, weights['h1']) + biases['h1'])
        layer2 = tf.nn.relu(tf.matmul(layer1 , weights['h2']) + biases['h2'])
        layer3 = tf.nn.relu(tf.matmul(layer2 , weights['h3']) + biases['h3'])
        out    = tf.matmul(layer3 , weights['out']) + biases['out']                   
        return tf.nn.softmax(out)  
   
  valid_prediction = predict(tf_valid_dataset,weights,biases)                         
  test_prediction  = predict(tf_test_dataset ,weights,biases)                            

In [308]:
num_steps = 1500001

startT = time.time()

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    choice = np.random.choice(train_labels.shape[0],batch_size)
    batch_data = train_dataset[choice, :]
    batch_labels = train_labels[choice, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

print('Elapsed Time: %.2f min'%( (time.time()-startT)/60.))

Initialized
Minibatch loss at step 0: 62644.042969
Minibatch accuracy: 7.0%
Validation accuracy: 19.4%
Minibatch loss at step 50000: 415.143158
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 100000: 281.922333
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 150000: 192.987427
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 200000: 133.205353
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 250000: 92.758568
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 300000: 65.211617
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 350000: 46.137466
Minibatch accuracy: 15.6%
Validation accuracy: 19.2%
Minibatch loss at step 400000: 33.056683
Minibatch accuracy: 18.0%
Validation accuracy: 21.3%
Minibatch loss at step 450000: 24.003407
Minibatch accuracy: 21.1%
Validation accuracy: 34.0%
Minibatch loss at step 500000: 17.489857
Minibatch 

In [283]:
batch_size = 128
l2_value = 0.005
n_hidden_1= 512 # 256 # relu layer
n_hidden_2= 512 # relu layer
dropout= 0.5
learning_rate=0.02

Minibatch loss at step 70000: 0.522530
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%
Test accuracy: 92.5%

SyntaxError: invalid syntax (<ipython-input-283-dbd2abc770c9>, line 8)

In [ ]:
batch_size = 128
l2_value = 0.007
n_hidden_1, n_hidden_2, n_hidden_3= 512, 256, 64
dropout= 0.9
learning_rate=0.005

Minibatch loss at step 300000: 0.559653
Minibatch accuracy: 97.7%
Validation accuracy: 87.5%
Test accuracy: 93.0%
Elapsed Time: 42.12 min

In [ ]:
batch_size = 128
l2_value = 0.005
n_hidden_1, n_hidden_2, n_hidden_3= 512, 256, 128
dropout= 0.8
learning_rate=0.003

Minibatch loss at step 400000: 0.496380
Minibatch accuracy: 97.7%
Validation accuracy: 87.6%
Test accuracy: 93.1%
Elapsed Time: 57.58 min

In [ ]:
batch_size = 128
l2_value = 0.004
n_hidden_1, n_hidden_2, n_hidden_3= 512, 256, 128
dropout= 0.80
learning_rate, decay_rate, decay_steps = 0.005, 0.7, 150000

Minibatch loss at step 500000: 0.419205
Minibatch accuracy: 97.7%
Validation accuracy: 87.7%
Test accuracy: 93.1%
Elapsed Time: 73.54 min


In [ ]:
batch_size = 128
l2_value = 0.003
n_hidden_1, n_hidden_2, n_hidden_3= 512, 256, 128
dropout= 0.80
learning_rate, decay_rate, decay_steps = 0.005, 0.7, 150000

Minibatch loss at step 500000: 0.429590
Minibatch accuracy: 99.2%
Validation accuracy: 87.5%
Test accuracy: 93.1%
Elapsed Time: 71.47 min
